In [2]:

import os
from pydantic import BaseModel, Extra, root_validator
from typing import Any, Dict, Optional
from time import sleep


def get_from_dict_or_env(data: Dict[str, Any], key: str, env_key: str, default: Optional[str] = None) -> str:
    """Get a value from a dictionary or an environment variable."""
    if key in data and data[key]:
        return data[key]
    elif env_key in os.environ and os.environ[env_key]:
        return os.environ[env_key]
    elif default is not None:
        return default
    else:
        raise ValueError(
            f"Did not find {key}, please add an environment variable"
            f" `{env_key}` which contains it, or pass"
            f"  `{key}` as a named parameter."
        )


# Imported from langchain
class WolframAlphaAPIWrapper(BaseModel):
    """Wrapper for Wolfram Alpha.

    Docs for using:

    1. Go to wolfram alpha and sign up for a developer account
    2. Create an app and get your APP ID
    3. Save your APP ID into WOLFRAM_ALPHA_APPID env variable
    4. pip install wolframalpha

    """

    wolfram_client: Any  #: :meta private:
    wolfram_alpha_appid: Optional[str] = None

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that api key and python package exists in environment."""
        wolfram_alpha_appid = get_from_dict_or_env(values, "wolfram_alpha_appid", "WOLFRAM_ALPHA_APPID")
        values["wolfram_alpha_appid"] = wolfram_alpha_appid

        try:
            import wolframalpha

        except ImportError:
            raise ImportError("wolframalpha is not installed. " "Please install it with `pip install wolframalpha`")
        client = wolframalpha.Client(wolfram_alpha_appid)
        values["wolfram_client"] = client

        return values

    def run(self, query: str) -> str:
        """Run query through WolframAlpha and parse result."""
        from urllib.error import HTTPError

        is_success = False  # added
        res = None
        for _ in range(20):
            try:
                res = self.wolfram_client.query(query)
                break
            except HTTPError:
                sleep(1)
            except Exception:
                return (
                    "Wolfram Alpha wasn't able to answer it. Please try a new query for wolfram or use python.",
                    is_success,
                )
        if res is None:
            return (
                "Wolfram Alpha wasn't able to answer it (may due to web error), you can try again or use python.",
                is_success,
            )

        try:
            if not res["@success"]:
                return (
                    "Your Wolfram query is invalid. Please try a new query for wolfram or use python.",
                    is_success,
                )
            assumption = next(res.pods).text
            answer = ""
            for r in res["pod"]:
                if r["@title"] == "Solution":
                    answer = r["subpod"]["plaintext"]
                if r["@title"] == "Results" or r["@title"] == "Solutions":
                    for i, sub in enumerate(r["subpod"]):
                        answer += f"ans {i}: " + sub["plaintext"] + "\n"
                    break
            if answer == "":
                answer = next(res.results).text

        except Exception:
            return (
                "Wolfram Alpha wasn't able to answer it. Please try a new query for wolfram or use python.",
                is_success,
            )

        if answer is None or answer == "":
            # We don't want to return the assumption alone if answer is empty
            return "No good Wolfram Alpha Result was found", is_success
        else:
            is_success = True
            return f"Assumption: {assumption} \nAnswer: {answer}", is_success


# imports and keys
import numpy as np
import pandas as pd

from flaml.autogen.agent import AssistantAgent, Agent, UserProxyAgent
from flaml import oai
from flaml.autogen.code_utils import execute_code

os.environ["WOLFRAM_ALPHA_APPID"] = open("wolfram.txt").read().strip()
config_list = config_list = [{
    'model' : 'gpt-4-0613',
    'api_key': open("key_e.txt").read().strip(),
}]

## Use Wolfram

In [3]:
sys_prompt = """You are an advanced AI with the capability to solve complex math problems.
Wolfram alpha is provided as an external service to help you solve math problems.

When the user gives a math problem, please use the most efficient way to solve the problem.
You are encouraged to use Wolfram alpha whenever it is possible during the solving process. For example, simplications, calculations, equation solving, etc.
However, if the operation requires little computation (very simple calculations, etc), you can also solve it directly.
Reply "TERMINATE" in the end when everything is done.
"""

func_config = {
    'functions':[
        {
            "name": "query_wolfram",
            "description": "Return the API query result from the Wolfram Alpha.",
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {
                        "type": "string",
                        "description": "The Wolfram Alpha code to be executed.",
                    }
                },
                "required": ["code"],
            },
        }
    ],
    'function_call':"auto",
}

# function to be passed
def query_wolfram(code):
    wolfram = WolframAlphaAPIWrapper()
    output, is_success = wolfram.run(code)
    if output == "":
        output = "Error: The wolfram query is invalid."
        is_success = False
    return output.strip()

chatbot = AssistantAgent("chatbot",sys_prompt,config_list=config_list, **func_config)
# user = Agent("user")
user = UserProxyAgent("user", human_input_mode="NEVER", functions={"query_wolfram": query_wolfram})

chatbot.receive(
    "Problem: Find all $x$ that satisfy the inequality $(2x+10)(x+3)<(3x+9)(x+8)$. Express your answer in interval notation.",
    user
)


user (to chatbot):
Problem: Find all $x$ that satisfy the inequality $(2x+10)(x+3)<(3x+9)(x+8)$. Express your answer in interval notation.

--------------------------------------------------------------------------------
chatbot (to user):
*****Calling function: query_wolfram*****
with arguments:  {
  "code": "solve (2x+10)(x+3)<(3x+9)(x+8) for x"
}
****************************************

--------------------------------------------------------------------------------
>>>>>>>> NO HUMAN INPUT RECEIVED. USING AUTO REPLY FOR THE USER...
user (to chatbot):
*****Response from calling function: query_wolfram*****
Assumption: solve (2 x + 10) (x + 3)<(3 x + 9) (x + 8) for x 
Answer: ans 0: x<-14
ans 1: x>-3
**************************************** 


--------------------------------------------------------------------------------
chatbot (to user):
The solution to the inequality $(2x+10)(x+3)<(3x+9)(x+8)$ is $x \in (-\infty, -14) \cup (-3, +\infty)$. 

TERMINATE

---------------------------

### Use Python

In [6]:

sys_prompt = """You are an advanced AI with the capability to solve complex math problems.
You can use Python code as an external computing resource. You can write python code to be executed. 
Note:
1. Your previous code will not be saved. So you need to define variables and functions again if you need them.
2. Your have to print the result of your code. Otherwise the result will not be returned.

When the user gives a math problem, please use the most efficient way to solve the problem.
You are encouraged to use Python whenever it is possible during the solving process. For example, simplications, calculations, equation solving, etc.
However, if the operation requires little computation (very simple calculations, etc), you can also solve it directly.

Reply a single "TERMINATE" in the one message when everything is done.
"""

func_config = {
    'functions':[
        {
            "name": "run_python_code",
            "description": "Return the execution result of the Python code. The printed output or the errors will be returned.",
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {
                        "type": "string",
                        "description": "The python code to be executed, with correct indentations.",
                    }
                },
                "required": ["code"],
            },
        }
    ],
    'function_call':"auto",
}

def run_python_code(code):
    def add_print_to_last_line(s):
        """Add print() to the last line of a string."""
        # 1. check if there is already a print statement
        if "print(" in s:
            return s
        # 2. extract the last line, enclose it in print() and return the new string
        lines = s.splitlines()
        last_line = lines[-1]
        if "\t" in last_line or "=" in last_line:
            return s
        if "=" in last_line:
            last_line = "print(" + last_line.split(" = ")[0] + ")"
            lines.append(last_line)
        else:
            lines[-1] = "print(" + last_line + ")"
        # 3. join the lines back together
        return "\n".join(lines)
    exitcode, logs, image = execute_code(add_print_to_last_line(code), use_docker=False)
    return logs.decode("utf-8")

def is_termination_msg(x):
    return lambda x: "TERMINATE" in x

chatbot = AssistantAgent("chatbot",sys_prompt,config_list=config_list, **func_config)
# user = Agent("user")
user = UserProxyAgent(
    "user", 
    human_input_mode="NEVER", 
    functions={"run_python_code": run_python_code},
)

chatbot.receive(
    "Problem: Find $x^3=125$",
    user
)

user (to chatbot):
Problem: Find $x^3=125$

--------------------------------------------------------------------------------
chatbot (to user):
*****Calling function: run_python_code*****
with arguments:  {
"code": "x = 125 ** (1/3)\nprint(x)"
}
****************************************

--------------------------------------------------------------------------------
>>>>>>>> NO HUMAN INPUT RECEIVED. USING AUTO REPLY FOR THE USER...
user (to chatbot):
*****Response from calling function: run_python_code*****
4.999999999999999

**************************************** 


--------------------------------------------------------------------------------
chatbot (to user):
The solution to the equation $x^3=125$ is $x=5$.
TERMINATE.

--------------------------------------------------------------------------------
>>>>>>>> NO HUMAN INPUT RECEIVED. USING AUTO REPLY FOR THE USER...
user (to chatbot):


--------------------------------------------------------------------------------
chatbot (to u

In [ ]:

class PythonKernel:
    def __init__(self) -> None:
        pass

    def exec_function(self,):
        pass

    